# Data Cleaning and Preparation
## UCI ML Online Retail Dataset

### Introduction
This notebook performs initial data cleaning and preparation on the Online Retail dataset. The cleaning process includes handling missing values, data type conversions, currency normalization, and basic validation checks.

### Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

# Set display options and suppress warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

### Load and Initial Data Inspection

In [8]:
# Read the Excel file
df = pd.read_excel('../data/online_retail.xlsx')

# Display basic information about the dataset
print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
print("\nDataset Info:")
df.info()

Dataset Shape: (541909, 8)

First few rows:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00      2.550   17850.000  United Kingdom  
1 2010-12-01 08:26:00      3.390   17850.000  United Kingdom  
2 2010-12-01 08:26:00      2.750   17850.000  United Kingdom  
3 2010-12-01 08:26:00      3.390   17850.000  United Kingdom  
4 2010-12-01 08:26:00      3.390   17850.000  United Kingdom  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       No

### Data Quality Assessment

In [9]:
# Check missing values
missing_values = df.isnull().sum()
missing_percentages = (missing_values / len(df)) * 100

print("Missing Values Analysis:")
for col in df.columns:
    print(f"{col}: {missing_values[col]} missing values ({missing_percentages[col]:.2f}%)")

# Check for duplicates
duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

# Basic statistics of numerical columns
print("\nNumerical Columns Statistics:")
print(df.describe())

Missing Values Analysis:
InvoiceNo: 0 missing values (0.00%)
StockCode: 0 missing values (0.00%)
Description: 1454 missing values (0.27%)
Quantity: 0 missing values (0.00%)
InvoiceDate: 0 missing values (0.00%)
UnitPrice: 0 missing values (0.00%)
CustomerID: 135080 missing values (24.93%)
Country: 0 missing values (0.00%)

Number of duplicate rows: 5268

Numerical Columns Statistics:
        Quantity                    InvoiceDate  UnitPrice  CustomerID
count 541909.000                         541909 541909.000  406829.000
mean       9.552  2011-07-04 13:34:57.156386048      4.611   15287.691
min   -80995.000            2010-12-01 08:26:00 -11062.060   12346.000
25%        1.000            2011-03-28 11:34:00      1.250   13953.000
50%        3.000            2011-07-19 17:17:00      2.080   15152.000
75%       10.000            2011-10-19 11:27:00      4.130   16791.000
max    80995.000            2011-12-09 12:50:00  38970.000   18287.000
std      218.081                            N

### Handle Missing Values and Invalid Entries

In [10]:
# Make a copy of the original dataframe
df_cleaned = df.copy()

# Remove rows with missing CustomerID (these are typically canceled orders)
df_cleaned = df_cleaned.dropna(subset=['CustomerID'])

# Convert CustomerID to integer type
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].astype(int)

# Remove rows with invalid quantities (<=0) or unit prices (<0)
df_cleaned = df_cleaned[df_cleaned['Quantity'] > 0]
df_cleaned = df_cleaned[df_cleaned['UnitPrice'] >= 0]

print("Rows before cleaning:", len(df))
print("Rows after cleaning:", len(df_cleaned))

Rows before cleaning: 541909
Rows after cleaning: 397924


### Data Type Conversions and Format Standardization

In [11]:
# Convert InvoiceDate to datetime
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

# Create additional date-related features
df_cleaned['Year'] = df_cleaned['InvoiceDate'].dt.year
df_cleaned['Month'] = df_cleaned['InvoiceDate'].dt.month
df_cleaned['Day'] = df_cleaned['InvoiceDate'].dt.day
df_cleaned['Hour'] = df_cleaned['InvoiceDate'].dt.hour

# Convert string columns to proper case
df_cleaned['Description'] = df_cleaned['Description'].str.strip().str.title()
df_cleaned['Country'] = df_cleaned['Country'].str.strip().str.title()

print("Updated DataFrame Info:")
df_cleaned.info()

Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 397924 entries, 0 to 541908
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  int64         
 7   Country      397924 non-null  object        
 8   Year         397924 non-null  int32         
 9   Month        397924 non-null  int32         
 10  Day          397924 non-null  int32         
 11  Hour         397924 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(4), int64(2), object(4)
memory usage: 33.4+ MB


### Add Derived Features

In [13]:
# Calculate total amount per transaction
df_cleaned['TotalAmount'] = df_cleaned['Quantity'] * df_cleaned['UnitPrice']

# Convert InvoiceNo to string type first
df_cleaned['InvoiceNo'] = df_cleaned['InvoiceNo'].astype(str)

# Create invoice type flag (C for canceled orders)
df_cleaned['IsOrder'] = ~df_cleaned['InvoiceNo'].str.contains('C', na=False)

print("Sample of derived features:")
print(df_cleaned[['InvoiceNo', 'TotalAmount', 'IsOrder']].head())

Sample of derived features:
  InvoiceNo  TotalAmount  IsOrder
0    536365       15.300     True
1    536365       20.340     True
2    536365       22.000     True
3    536365       20.340     True
4    536365       20.340     True


### Data Validation and Quality Checks

In [14]:
def validate_dataset(df):
    """
    Perform final validation checks on the cleaned dataset
    Returns dictionary with validation results
    """
    validation_results = {
        'no_missing_values': df.isnull().sum().sum() == 0,
        'positive_quantities': (df['Quantity'] > 0).all(),
        'valid_prices': (df['UnitPrice'] >= 0).all(),
        'valid_customer_ids': df['CustomerID'].notna().all(),
        'valid_total_amounts': (df['TotalAmount'] >= 0).all()
    }
    
    return validation_results

validation_results = validate_dataset(df_cleaned)
print("Validation Results:")
for check, result in validation_results.items():
    print(f"{check}: {'Passed' if result else 'Failed'}")

# Basic statistics of the cleaned dataset
print("\nCleaned Dataset Statistics:")
print(df_cleaned.describe())

Validation Results:
no_missing_values: Passed
positive_quantities: Passed
valid_prices: Passed
valid_customer_ids: Passed
valid_total_amounts: Passed

Cleaned Dataset Statistics:
        Quantity                    InvoiceDate  UnitPrice  CustomerID  \
count 397924.000                         397924 397924.000  397924.000   
mean      13.022  2011-07-10 23:43:36.912475648      3.116   15294.315   
min        1.000            2010-12-01 08:26:00      0.000   12346.000   
25%        2.000            2011-04-07 11:12:00      1.250   13969.000   
50%        6.000            2011-07-31 14:39:00      1.950   15159.000   
75%       12.000            2011-10-20 14:33:00      3.750   16795.000   
max    80995.000            2011-12-09 12:50:00   8142.750   18287.000   
std      180.420                            NaN     22.097    1713.170   

            Year      Month        Day       Hour  TotalAmount  
count 397924.000 397924.000 397924.000 397924.000   397924.000  
mean    2010.934      7.

### Save Cleaned Dataset

In [15]:
# Save cleaned dataset to CSV
output_path = '../data/online_retail_cleaned.csv'
df_cleaned.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to: {output_path}")

# Display final dataset shape and memory usage
print(f"\nFinal dataset shape: {df_cleaned.shape}")
print(f"Memory usage: {df_cleaned.memory_usage().sum() / 1024**2:.2f} MB")


Cleaned dataset saved to: ../data/online_retail_cleaned.csv

Final dataset shape: (397924, 14)
Memory usage: 36.81 MB
